In [ ]:
%cd ..

In [8]:
CONFIG_DICT = {
    'configuration': 'LF',
    'path_train': 'data/Train',
    'path_test': 'data/Test',
    'win_size': 100
}

In [5]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MaxNLocator

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, brier_score_loss

import xgboost as xgb

from scipy.stats import kurtosis, skew, trim_mean, iqr

#from sktime.classification.feature_based import Catch22Classifier
from sktime.datatypes._panel._convert import from_3d_numpy_to_nested

#from tslearn.early_classification import NonMyopicEarlyClassifier
#from tslearn.utils import to_time_series_dataset, to_time_series

import warnings
warnings.filterwarnings("ignore")

In [6]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 16, 10

In [ ]:
assert os.listdir(CONFIG_DICT['path_train']) is not None

In [ ]:
df = pd.concat(
    [pd.read_csv(os.path.join(CONFIG_DICT['path_train'], p)) for p in os.listdir(CONFIG_DICT['path_train'])]
)
df_test_temp = pd.concat(
    [pd.read_csv(os.path.join(CONFIG_DICT['path_test'], p)) for p in os.listdir(CONFIG_DICT['path_test'])]
    )

In [10]:
configuration = CONFIG_DICT['configuration']

for c in df.columns:
    if configuration:
        if not (c.startswith(f'Mag{configuration}') or c.startswith('Indoor') or c =='Patient' or c.startswith('T') or c=='Date'):
            df.drop(columns=c, inplace=True)
            df_test_temp.drop(columns=c, inplace=True)
    else:
        if not (c.startswith('Mag') or c.startswith('Indoor') or c=='Patient' or c.startswith('T') or c=='Date'):
            df.drop(columns=c, inplace=True)
            df_test_temp.drop(columns=c, inplace=True)

for col in df.columns:
  if df[col].dtypes=='float64':
    df[col] = df[col].astype('float32')
    df_test_temp[col] = df_test_temp[col].astype('float32')
  if df[col].dtype=='int64':
    df[col] = df[col].astype('int16')
    df_test_temp[col] = df_test_temp[col].astype('int16')

df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
df_test_temp['Timestamp'] = pd.to_datetime(df_test_temp['Timestamp'], unit='s')

#sorted(df_test_temp['Patient'].unique())
not_cols = set(['Timestamp', 'Date', 'Patient'])
allCols = set(df.columns.to_list())

FEATURE_COLS = sorted(list(set.difference(allCols, not_cols)))
FEATURE_COLS

['Indoor', 'MagLF_Norm', 'MagLF_x', 'MagLF_y', 'MagLF_z']

In [12]:
df_train = pd.DataFrame(columns=FEATURE_COLS)

In [13]:
for k, group in df.groupby('Patient'):
  print('Processing patient: ', k)
  #print(type(group))
  if k not in [6001, 6002, 6003]:
    for ts in group.Timestamp.unique():
      if group[group['Timestamp']==ts]['Timestamp'].value_counts().values in [12800, 12799, 12801]:
        data = group[group['Timestamp']==ts][FEATURE_COLS]
        df_train = pd.concat([df_train, data])
  else:
    data = group[~(group['Date'].isin(['13/04/2022 10:05:37','13/04/2022 18:34:28','14/06/2022 11:16:13',
                                      '14/06/2022 14:20:03', '14/04/2022 07:58:00', '14/04/2022 16:22:01']))][FEATURE_COLS]
    df_train = df_train.append(data)

Processing patient:  1004
Processing patient:  3000


In [14]:
df_train['series_id'] = np.arange(len(df_train)) // CONFIG_DICT['win_size'] + 1
y_train = df_train[['series_id', 'Indoor']]

In [15]:
df.Indoor.sum()/len(df)

0.6451544088909752

In [17]:
count_train, count_test, count_val = 0, 0, 0

for t in df_train.series_id.value_counts().values:
  if t!=CONFIG_DICT['win_size']:
    count_train+=t

if count_train!=0:
  df_train = df_train.iloc[:-count_train]

In [18]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()

#df_train[FEATURE_COLS[1:]] = scl.fit_transform(df_train[FEATURE_COLS[1:]])
y_train = df_train[['series_id', 'Indoor']]

In [19]:
train_sequences = []
labels = []
for series_id, group in tqdm(df_train.groupby('series_id'), position=0, leave=True):
  sequence_features = group[FEATURE_COLS[1:]].to_numpy()
  labels.append(y_train[y_train.series_id==series_id].iloc[0].Indoor)
  train_sequences.append(sequence_features)

train_sequences = np.array(train_sequences)
train_sequences = np.swapaxes(train_sequences, 1,2)
train_sequences.shape

100%|██████████| 18432/18432 [00:54<00:00, 340.47it/s]


(18432, 4, 100)

In [20]:
train_df = from_3d_numpy_to_nested(train_sequences, column_names=FEATURE_COLS[1:], cells_as_numpy=True)
y_train = pd.DataFrame(labels)

VANILLA STATISTICS

* mean,
* median,
* std dev,
* mean absolute deviation
* percentiles (1,10,25,50,75,99)
* iqr
* trimmed mean (0.125)


TIME DOMAIN

* root mean square,
* variance,
* kurtosis,
* skewness
* correlation between each pair of magnetometer

FREQUENCY DOMAIN

* the dominant frequency
* the power at the dominant frequency

In [21]:
def calc_corr(data,axis,sensor):
    if axis=='xy':
        corr_xy = np.corrcoef(data[f'Mag{sensor}_{axis[0]}'], data[f'Mag{sensor}_{axis[1]}'])
        return corr_xy[0][1]
    elif axis=='xz':
        corr_xz = np.corrcoef(data[f'Mag{sensor}_{axis[0]}'], data[f'Mag{sensor}_{axis[1]}'])
        return corr_xz[0][1]
    elif axis=='yz':
        corr_yz = np.corrcoef(data[f'Mag{sensor}_{axis[0]}'], data[f'Mag{sensor}_{axis[1]}'])
        return corr_yz[0][1]

In [22]:
def extract_dominant(data):
    time_step = 1/100

    Hn = np.fft.fft(data)
    freqs = np.fft.fftfreq(len(Hn), time_step)
    idx = np.argsort(np.abs(Hn))

    fs = freqs[idx]
    return (fs[0], fs[1])

In [23]:
def extract_power(data):
    ps = np.abs(np.fft.fft(data))**2
    time_step = 1/100

    freqs = np.fft.fftfreq(len(data), time_step)
    idx = np.argsort(freqs)
    ps = ps[idx]

    return (ps[0], ps[1])

In [24]:
def transform_dataset(df):
    sensors = ['LB', 'LF', 'RF', 'WR'] if not configuration else [configuration]
    print(sensors)

    time_feat_1 =  {
        'mean': np.array([np.array([np.mean(row[c]) for c in row.keys()]) for _,row in df.iterrows()]),
        'median': np.array([np.array([np.median(row[c]) for c in row.keys()]) for _,row in df.iterrows()]),
        'std': np.array([np.array([np.std(row[c]) for c in row.keys()]) for _,row in df.iterrows()]),
        'mad': np.array([np.array([np.mean(np.abs(row[c]-np.mean(row[c]))) for c in row.keys()]) for _,row in df.iterrows()]),
        'rms': np.array([np.array([np.sqrt(np.mean(row[c]**2)) for c in row.keys()]) for _,row in df.iterrows()]),
        'vars': np.array([np.array([np.var(row[c]) for c in row.keys()]) for _,row in df.iterrows()]),
        'kurts': np.array([np.array([kurtosis(row[c]) for c in row.keys()]) for _,row in df.iterrows()]),
        'skews': np.array([np.array([skew(row[c]) for c in row.keys()]) for _,row in df.iterrows()]),
        'p_1': np.array([np.array([np.percentile(row[c], 1) for c in row.keys()]) for _,row in df.iterrows()]),
        'p_10': np.array([np.array([np.percentile(row[c], 10) for c in row.keys()]) for _,row in df.iterrows()]),
        'p_25': np.array([np.array([np.percentile(row[c], 25) for c in row.keys()]) for _,row in df.iterrows()]),
        'p_50': np.array([np.array([np.percentile(row[c], 50) for c in row.keys()]) for _,row in df.iterrows()]),
        'p_75': np.array([np.array([np.percentile(row[c], 75) for c in row.keys()]) for _,row in df.iterrows()]),
        'p_99': np.array([np.array([np.percentile(row[c], 99) for c in row.keys()]) for _,row in df.iterrows()]),
        'iqr': np.array([np.array([iqr(row[c]) for c in row.keys()]) for _,row in df.iterrows()]),
        'trim_mean125': np.array([np.array([trim_mean(row[c], 0.125) for c in row.keys()]) for _,row in df.iterrows()])
    }

    time_feat_2 = {
        'c_xy': np.array([np.array([calc_corr(row, 'xy', s) for s in sensors]) for _,row in df.iterrows()]),
        'c_xz': np.array([np.array([calc_corr(row, 'xz', s) for s in sensors]) for _,row in df.iterrows()]),
        'c_yz': np.array([np.array([calc_corr(row, 'yz', s) for s in sensors]) for _,row in df.iterrows()])
    }

    freq_feat = {
        'f0': np.array([np.array([extract_dominant(row[c])[0] for c in row.keys()]) for _,row in df.iterrows()]),
        'f1': np.array([np.array([extract_dominant(row[c])[1] for c in row.keys()]) for _,row in df.iterrows()]),
        'pwr0': np.array([np.array([extract_power(row[c])[0] for c in row.keys()]) for _,row in df.iterrows()]),
        'pwr1': np.array([np.array([extract_power(row[c])[1] for c in row.keys()]) for _,row in df.iterrows()]),

    }

    time_df_1 = pd.concat([pd.DataFrame(time_feat_1[tf],
        columns=[f'{tf}_dim{i}' for i in range(4*len(sensors))]) for tf in tqdm(time_feat_1.keys())], axis=1)
    time_df_2 = pd.concat([pd.DataFrame(time_feat_2[tf],
        columns=[f'{tf}_dim{i}' for i in range(len(sensors))]) for tf in tqdm(time_feat_2.keys())], axis=1)
    freq_df = pd.concat([pd.DataFrame(freq_feat[ff],
        columns=[f'{ff}_dim{i}' for i in range(4*len(sensors))]) for ff in tqdm(freq_feat.keys())], axis=1)

    X = pd.concat([time_df_1, time_df_2, freq_df], axis=1)

    return X

In [25]:
X_train = transform_dataset(train_df)
X_train_scl = scl.fit_transform(X_train)

['LF']


100%|██████████| 4/4 [00:00<00:00, 4033.95it/s]


In [26]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit_transform(X_train_scl)
var_wanted = 0.85

total = sum(pca.explained_variance_)
k_pca = 0
current_variance = 0
while current_variance/total < var_wanted:
    current_variance += pca.explained_variance_[k_pca]
    k_pca = k_pca + 1

print(k_pca, f"features explain around {var_wanted*100}% of the variance.")

13 features explain around 85.0% of the variance.


In [27]:
pca = PCA(n_components=k_pca)
X_train_pca = pca.fit_transform(X_train_scl)

In [28]:
def plot_auroc(y_test, y_pred_proba, show=False):
    if (y_test.squeeze().sum()/y_test.shape[0]<=0.05) or (y_test.squeeze().sum()/y_test.shape[0]>=0.95):
        print('Test set too unbalanced. No AUROC is provided')
    else:
        auc = roc_auc_score(y_test.astype(dtype=np.int8), y_pred_proba)
        print('AUROC Score: ', auc)
        if show:
            plt.figure(figsize=(8,5))
            fpr, tpr, _ = roc_curve(y_test.astype(dtype=np.int8), y_pred_proba)
            plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
            plt.legend(loc=4)
            #plt.show()

In [49]:
def clf_report(clf, X_test, y_test):
    y_pred = clf.predict(X_test)

    y_pred_proba = clf.predict_proba(X_test)[:,1]
    print(f'Brier Score: {brier_score_loss(y_test, y_pred_proba)}')
    plot_auroc(y_test, y_pred_proba)
    print('Classification report: ')
    print(classification_report(y_test, y_pred))
    #with open(file_to_write, 'a') as f:
    #  f.write(f'Brier Score: {brier_score_loss(y_test, y_pred_proba)}')
    #  f.write('\n')
    #  f.write(f'AUROC Score: {auc}')
    #  f.write('\n')
    #  f.write(classification_report)

    print('-'*20)

In [50]:
isTest = False

def fit_models(X, X_scaled, y, fit=True):
    if not isTest:
        ''' Random Forest '''
        rfc = RandomForestClassifier(
            n_jobs=-1,
            random_state=42,
            n_estimators=200,
            max_features=0.5,
            max_depth=4
        )

        ''' XGB '''
        bst = xgb.XGBClassifier(
                subsample = 0.5,
                min_child_weight = 1,
                max_depth=2,
                objective = 'binary:logistic'
        )
        
    clfs = [
        (rfc, 'RF'),
        (bst, 'XGB')

    ]

    if fit:
        for c in clfs:
            if c[1]=='XGB':
                c[0].fit(X_scaled, y)
            else:
                c[0].fit(X, y)

    return clfs

In [56]:
y_train = y_train.astype('int8')

In [57]:
clf = fit_models(X_train, X_train_scl, y_train)
clf_pca = fit_models(X_train_pca, X_train_pca, y_train)

[02:05:00] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:05:03] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [61]:
for p_test in df_test_temp.Patient.unique():

  df_test = pd.DataFrame(columns=FEATURE_COLS)

  for k, group in df_test_temp[df_test_temp['Patient']==p_test].groupby('Patient'):
    print('Processing patient: ', k)
    #print(type(group))
    if k not in [6001, 6002, 6003]:
      for ts in group.Timestamp.unique():
        if group[group['Timestamp']==ts]['Timestamp'].value_counts().values in [12800, 12799, 12801]:
          data = group[group['Timestamp']==ts][FEATURE_COLS]
          df_test = pd.concat([df_test, data])
    else:
      data = group[~(group['Date'].isin(['13/04/2022 10:05:37','13/04/2022 18:34:28','14/06/2022 11:16:13',
                                        '14/06/2022 14:20:03', '14/04/2022 07:58:00', '14/04/2022 16:22:01']))][FEATURE_COLS]
      df_test = pd.concat([df_test, data])

  df_test['series_id'] = np.arange(len(df_test)) // CONFIG_DICT['win_size'] + 1
  y_test = df_test[['series_id', 'Indoor']]
  #y_test = df_test['Indoor']
  count_test = 0

  for t in df_test.series_id.value_counts().values:
    if t!=CONFIG_DICT['win_size']:
      count_test+=t

  if count_test!=0:
    df_test = df_test.iloc[:-count_test]

  #if to_scl:
  #df_test[FEATURE_COLS[1:]] = scl.transform(df_test[FEATURE_COLS[1:]])
  y_test = df_test[['series_id', 'Indoor']]

  test_sequences = []
  labels_test = []

  for series_id, group in tqdm(df_test.groupby('series_id'), position=0, leave=True):
    sequence_features = group[FEATURE_COLS[1:]].to_numpy()
    labels_test.append(y_test[y_test.series_id==series_id].iloc[0].Indoor)
    test_sequences.append(sequence_features)

  test_sequences = np.array(test_sequences)
  test_sequences = np.swapaxes(test_sequences,1,2)

  print(test_sequences.shape)

  test_df = from_3d_numpy_to_nested(test_sequences, column_names=FEATURE_COLS[1:], cells_as_numpy=True)
  y_test = pd.DataFrame(labels_test)
  print(test_sequences.shape)
  X_test = transform_dataset(test_df)
  X_test_scl = scl.transform(X_test)
  print(f'Performances for subject: {p_test}\n')

  X_test_pca = pca.transform(X_test_scl)

  var_exp = pca.explained_variance_ratio_.cumsum()
  var_exp = var_exp*100
  #plt.bar(range(k), var_exp)

  for c,c_pca in zip(clf, clf_pca):
      print(c[1])
      if c[1]=='XGB':
        clf_report(c[0], X_test_scl, y_test)
      else:
        clf_report(c[0], X_test, y_test)
      clf_report(c_pca[0], X_test_pca, y_test)

Processing patient:  2007


100%|██████████| 9856/9856 [00:17<00:00, 563.54it/s]


(9856, 4, 100)
(9856, 4, 100)
['LF']


100%|██████████| 4/4 [00:00<00:00, 4001.24it/s]


Performances for subject: 2007

RF
Brier Score: 0.07893348708949222
Test set too unbalanced. No AUROC is provided
Classification report: 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.95      0.97      9856

    accuracy                           0.95      9856
   macro avg       0.50      0.47      0.49      9856
weighted avg       1.00      0.95      0.97      9856

--------------------
Brier Score: 0.05047747043976902
Test set too unbalanced. No AUROC is provided
Classification report: 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.95      0.98      9856

    accuracy                           0.95      9856
   macro avg       0.50      0.48      0.49      9856
weighted avg       1.00      0.95      0.98      9856

--------------------
XGB
Brier Score: 0.04719168879699655
Test set too unbalanced. N